# Tutorial 01 - Thermal Block

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.fem
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import plotly.graph_objects as go  # noqa: F401
import slepc4py  # noqa: F401
import ufl

In [ ]:
import rbnicsx.backends
import rbnicsx.online
import rbnicsx.test

In [ ]:
%load_ext nbvalx

In [ ]:
%register_run_if_allowed_tags \
    reduced_basis_inefficient, reduced_basis_efficient, \
    pod_galerkin_inefficient, pod_galerkin_efficient

In [ ]:
%register_run_if_current_tag reduced_basis_inefficient

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
gmsh.initialize()
gmsh.model.add("thermal_block")

In [ ]:
p0 = gmsh.model.geo.addPoint(-1.0, -1.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, -1.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(-1.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
outer_boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])

In [ ]:
p4 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p5 = gmsh.model.geo.addPoint(0.5, 0.0, 0.0, mesh_size)
p6 = gmsh.model.geo.addPoint(-0.5, 0.0, 0.0, mesh_size)
c0 = gmsh.model.geo.addCircleArc(p5, p4, p6)
c1 = gmsh.model.geo.addCircleArc(p6, p4, p5)
interface = gmsh.model.geo.addCurveLoop([c0, c1])

In [ ]:
outer_subdomain = gmsh.model.geo.addPlaneSurface([outer_boundary, interface])
inner_subdomain = gmsh.model.geo.addPlaneSurface([interface])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1, l3], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(2, [inner_subdomain], 1)
gmsh.model.addPhysicalGroup(2, [outer_subdomain], 2)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        self._trial = u
        self._test = v
        # Define measures for integration of forms
        dx = ufl.Measure("dx")(subdomain_data=subdomains)
        ds = ufl.Measure("ds")(subdomain_data=boundaries)
        self._dx = dx
        self._ds = ds
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = mu_symb[0] * ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(1) + ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(2)
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        f = ufl.inner(mu_symb[1], v) * ds(1)
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        zero = petsc4py.PETSc.ScalarType(0)
        facets_top = boundaries.indices[boundaries.values == 3]
        bdofs_V_top = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_top)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V_top, V)]
        self._bcs = bcs

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def mu_symb(self) -> rbnicsx.backends.SymbolicParameters:
        """Return the symbolic parameters of the problem."""
        return self._mu_symb

    @property
    def trial_and_test(self) -> typing.Tuple[ufl.Argument, ufl.Argument]:
        """Return the UFL arguments used in the construction of the forms."""
        return self._trial, self._test

    @property
    def measures(self) -> typing.Tuple[ufl.Measure, ufl.Measure]:
        """Return the UFL measures used in the construction of the forms."""
        return self._dx, self._ds

    @property
    def bilinear_form(self) -> ufl.Form:
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.Function(self._V)
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([8.0, -1.0])
solution = problem.solve(mu_solve)

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution")

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
class StabilityFactorEvaluation(object):
    """Define the eigenvalue problem to compute the coercivity constant, and solve it with EPS."""

    def __init__(self, problem: problem) -> None:
        # Get function space from the high fidelity problem
        V = problem.function_space
        self._V = V
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Get measures from the high fidelity problem
        dx, _ = problem.measures
        # Store symbolic parameters from problem
        self._mu_symb = problem.mu_symb
        # Define bilinear forms of the eigenvalue problem
        a = problem.bilinear_form
        b = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        self._b = a
        self._b_cpp = dolfinx.fem.form(b)
        # Define restriction that throws away DOFs associated to Dirichlet boundary conditions
        bcs = problem.boundary_conditions
        bcs_dof_indices = np.hstack([bc.dof_indices()[0] for bc in bcs])
        dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
        restriction = multiphenicsx.fem.DofMapRestriction(V.dofmap, np.setdiff1d(dofs_V, bcs_dof_indices))
        self._restriction = restriction

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the eigenvalue problem."""
        return self._V

    @property
    def bilinear_form_left(self) -> ufl.Form:
        """Return the bilinear form of the left-hand side of the eigenvalue problem."""
        return self._a

    @property
    def bilinear_form_right(self) -> ufl.Form:
        """Return the bilinear form of the right-hand side of the eigenvalue problem."""
        return self._f

    @property
    def restriction(self) -> multiphenicsx.fem.DofMapRestriction:
        """Return restriction that throws away DOFs associated to homogenous Dirichlet boundary conditions."""
        return self._restriction

    def _assemble_matrix_left(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = multiphenicsx.fem.assemble_matrix(self._a_cpp, restriction=(self._restriction, self._restriction))
        A.assemble()
        return A

    def _assemble_matrix_right(self) -> petsc4py.PETSc.Mat:
        """Assemble the right-hand side matrix."""
        B = multiphenicsx.fem.assemble_matrix(self._b_cpp, restriction=(self._restriction, self._restriction))
        B.assemble()
        return B

    def evaluate(self, mu: np.typing.NDArray[np.float64]) -> petsc4py.PETSc.RealType:
        """Assign the provided parameters value and solve the eigenvalue problem."""
        self._mu_symb.value[:] = mu
        return self._evaluate()

    def _evaluate(self) -> petsc4py.PETSc.RealType:
        """Solve the eigenvalue problem with EPS."""
        A = self._assemble_matrix_left()
        B = self._assemble_matrix_right()
        eps = slepc4py.SLEPc.EPS().create(mesh.comm)
        eps.setOperators(A, B)
        eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GNHEP)
        eps.setDimensions(1, petsc4py.PETSc.DECIDE, petsc4py.PETSc.DECIDE)
        eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
        eps.setTarget(1.e-5)
        eps.getST().setType(slepc4py.SLEPc.ST.Type.SINVERT)
        eps.getST().getKSP().setType("preonly")
        eps.getST().getKSP().getPC().setType("lu")
        eps.getST().getKSP().getPC().setFactorSolverType("mumps")
        eps.solve()
        assert eps.getConverged() >= 1
        eigv = eps.getEigenvalue(0)
        r, i = eigv.real, eigv.imag
        assert np.isclose(i, 0.0)
        return r

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
stability_factor_evaluation = StabilityFactorEvaluation(problem)

In [ ]:
%%run_if reduced_basis_inefficient
stability_factor_lower_bound = stability_factor_evaluation

In [ ]:
%%run_if reduced_basis_inefficient
stability_factor = stability_factor_evaluation.evaluate(mu_solve)
print(f"Stability factor at {mu_solve} is {stability_factor}")

## 3. Reduced problem definition

In [ ]:
%%run_if reduced_basis_efficient
class StabilityFactorLowerBound(object):
    """Evaluate a lower bound of the stability factor using the min-theta method."""

    def evaluate(self, mu: np.typing.NDArray[np.float64]) -> petsc4py.PETSc.RealType:
        """Evaluate the stability factor using the min-theta method."""
        return min(mu[0], 1)

In [ ]:
%%run_if reduced_basis_efficient
stability_factor_lower_bound = StabilityFactorLowerBound()

In [ ]:
%%run_if reduced_basis_efficient
stability_factor_exact = stability_factor_evaluation.evaluate(mu_solve)
stability_factor_min_theta = stability_factor_lower_bound.evaluate(mu_solve)
print(f"Stability factor at {mu_solve} is {stability_factor_exact}")
print(f"Min-theta approximation of the stability factor at {mu_solve} is {stability_factor_min_theta}")
assert (
    stability_factor_min_theta <= stability_factor_exact
    or np.isclose(stability_factor_min_theta, stability_factor_exact))

In [ ]:
%%run_if pod_galerkin_inefficient, reduced_basis_inefficient
class ProjectionBasedInefficientReducedProblem(object):
    """Define a linear projection-based problem, and solve it with KSP."""

    def __init__(self, problem: Problem) -> None:
        # Define basis functions storage
        basis_functions = rbnicsx.backends.FunctionsList(problem.function_space)
        self._basis_functions = basis_functions
        # Store symbolic parameters from problem
        self._mu_symb = problem.mu_symb
        # Store forms of the problem
        self._a_action = rbnicsx.backends.bilinear_form_action(problem.bilinear_form)
        self._f_action = rbnicsx.backends.linear_form_action(problem.linear_form)
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Define inner product using the H^1 seminorm.
        inner_product = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
        self._inner_product = inner_product
        self._inner_product_action = rbnicsx.backends.bilinear_form_action(
            inner_product, part="real")
        # Define the energy inner product
        energy_inner_product = 0.5 * (problem.bilinear_form + ufl.adjoint(problem.bilinear_form))
        self._energy_inner_product = energy_inner_product
        self._energy_inner_product_action = rbnicsx.backends.bilinear_form_action(
            energy_inner_product, part="real")

    @property
    def basis_functions(self) -> rbnicsx.backends.FunctionsList:
        """Return the basis functions of the reduced problem."""
        return self._basis_functions

    @property
    def inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product

    @property
    def inner_product_action(self) -> typing.Callable:
        """
        Return the action of the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product_action

    @property
    def energy_inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the energy inner product associated to this reduced problem.

        For elliptic coercive problems this inner product is used to evaluate the norm of the error.
        """
        return self._energy_inner_product

    def _assemble_matrix(self, N: int) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side online matrix of dimension N."""
        return rbnicsx.backends.project_matrix(self._a_action, self._basis_functions[:N])

    def _assemble_vector(self, N: int) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side online vector of dimension N."""
        return rbnicsx.backends.project_vector(self._f_action, self._basis_functions[:N])

    def solve(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Assign the provided parameters value and solve the online problem."""
        self._mu_symb.value[:] = mu
        return self._solve(N)

    def _solve(self, N: int) -> petsc4py.PETSc.Vec:
        """Solve the linear online problem with KSP."""
        reduced_solution = rbnicsx.online.create_vector(N)
        if N > 0:
            A = self._assemble_matrix(N)
            F = self._assemble_vector(N)
            ksp = petsc4py.PETSc.KSP()
            ksp.create(mesh.comm)
            ksp.setOperators(A)
            ksp.setType("preonly")
            ksp.getPC().setType("lu")
            ksp.setFromOptions()
            ksp.solve(F, reduced_solution)
        return reduced_solution

    def reconstruct_solution(self, reduced_solution: petsc4py.PETSc.Vec) -> dolfinx.fem.Function:
        """Reconstructed reduced solution on the high fidelity space."""
        return self.basis_functions[:reduced_solution.size] * reduced_solution

    def compute_pointwise_error(
        self, solution: dolfinx.fem.Function, reduced_solution: petsc4py.PETSc.Vec
    ) -> dolfinx.fem.Function:
        """Compute the pointwise error."""
        reconstructed_reduced_solution = self.reconstruct_solution(reduced_solution)
        assert solution.function_space == reconstructed_reduced_solution.function_space
        error = dolfinx.fem.Function(solution.function_space)
        with error.vector.localForm() as error_local, solution.vector.localForm() as solution_local, \
                reconstructed_reduced_solution.vector.localForm() as reduced_solution_local:
            error_local[:] = solution_local.array - reduced_solution_local.array
        return error

    def compute_error_norm(self, error: dolfinx.fem.Function) -> petsc4py.PETSc.RealType:
        """Compute the norm of the error using the natural inner product."""
        return np.sqrt(self._inner_product_action(error)(error))

    def compute_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], error: dolfinx.fem.Function
    ) -> petsc4py.PETSc.RealType:
        """Compute the norm of the error using the energy inner product."""
        self._mu_symb.value[:] = mu
        return np.sqrt(self._energy_inner_product_action(error)(error))

In [ ]:
%%run_if pod_galerkin_inefficient
class PODGInefficientReducedProblem(ProjectionBasedInefficientReducedProblem):
    """Specialize the projection-based problem to basis generation with POD."""

    pass  # No further specialization needed

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
class RieszRepresentationProblem(object):
    """Define the Riesz representation problem, and solve it with KSP."""

    def __init__(self, problem: Problem, reduced_problem: object) -> None:
        # Get function space from the high fidelity problem
        V = problem.function_space
        self._V = V
        # Assemble the inner product matrix
        X = dolfinx.fem.assemble_matrix(
            dolfinx.fem.form(reduced_problem.inner_product_form), bcs=problem.boundary_conditions)
        X.assemble()
        self._inner_product_matrix = X
        # Determine DOFs associated to Dirichlet boundary conditions
        self._bcs_dof_indices = np.hstack([bc.dof_indices()[0] for bc in problem.boundary_conditions])

    def _assemble_vector(self, form_cpp: dolfinx.fem.FormMetaClass) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.assemble_vector(form_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        with F.localForm() as F_local:
            F_local[self._bcs_dof_indices] = np.zeros_like(self._bcs_dof_indices)
        return F

    def represent(self, form_cpp: dolfinx.fem.FormMetaClass) -> dolfinx.fem.Function:
        """Solve the Riesz representation problem with KSP."""
        F = self._assemble_vector(form_cpp)
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(self._inner_product_matrix)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        representer = dolfinx.fem.Function(self._V)
        ksp.solve(F, representer.vector)
        representer.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return representer

In [ ]:
%%run_if reduced_basis_inefficient
class RBInefficientReducedProblem(ProjectionBasedInefficientReducedProblem):
    """Specialize the projection-based problem to basis generation with RB."""

    def __init__(self, problem: Problem, stability_factor_lower_bound: object) -> None:
        super().__init__(problem)
        # Prepare storage for Riesz representation
        self._riesz_representation_problem = RieszRepresentationProblem(problem, self)
        # Store object that evaluates the stability factor lower bound
        self._stability_factor_lower_bound = stability_factor_lower_bound
        # Store helper form for residual norm evaluation
        u, _ = problem.trial_and_test
        residual_form_placeholder = dolfinx.fem.Function(problem.function_space)
        residual_form = problem.linear_form - ufl.replace(problem.bilinear_form, {u: residual_form_placeholder})
        self._residual_form_cpp = dolfinx.fem.form(residual_form)
        self._residual_form_placeholder = residual_form_placeholder

    def get_residual_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Compute the squared natural norm of the residual."""
        self._mu_symb.value[:] = mu
        reconstructed_solution = self.reconstruct_solution(reduced_solution)
        with reconstructed_solution.vector.localForm() as reconstructed_solution_local, \
                self._residual_form_placeholder.vector.localForm() as residual_form_placeholder_local:
            reconstructed_solution_local.copy(residual_form_placeholder_local)
        riesz = self._riesz_representation_problem.represent(self._residual_form_cpp)
        return np.sqrt(self._inner_product_action(riesz)(riesz))

    def estimate_error_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate the norm of the error using the natural inner product."""
        return self.get_residual_norm(mu, reduced_solution) / (
            self._stability_factor_lower_bound.evaluate(mu))

    def estimate_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate the norm of the error using the energy inner product."""
        return self.get_residual_norm(mu, reduced_solution) / np.sqrt(
            self._stability_factor_lower_bound.evaluate(mu))

In [ ]:
%%run_if pod_galerkin_efficient, reduced_basis_efficient
class ProjectionBasedReducedProblem(object):
    """Define a linear projection-based problem, and solve it with KSP."""

    def __init__(self, problem: Problem) -> None:
        # Define basis functions storage
        basis_functions = rbnicsx.backends.FunctionsList(problem.function_space)
        self._basis_functions = basis_functions
        # Get trial and test functions from the high fidelity problem
        u, v = problem.trial_and_test
        # Get measures from the high fidelity problem
        dx, ds = problem.measures
        # Store symbolic parameters from problem
        self._mu_symb = problem.mu_symb
        # Define separable expansion of the bilinear form of the high fidelity problem,
        # and storage for the corresponding pre-assembled reduced operators.
        thetas_a = (
            lambda mu: mu[0],
            lambda mu: 1
        )
        self._thetas_a = thetas_a
        operators_a = (
            ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(1),
            ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(2)
        )
        self._operators_a = operators_a
        self._operators_a_action = tuple(
            rbnicsx.backends.bilinear_form_action(operator_a) for operator_a in operators_a)
        assert len(thetas_a) == len(operators_a)
        self._Q_a = len(thetas_a)
        reduced_operators_a: typing.Dict[int, rbnicsx.online.TensorsList] = dict()
        self._reduced_operators_a = reduced_operators_a
        # Define separable expansion of the linear form of the high fidelity problem,
        # and storage for the corresponding pre-assembled reduced operators.
        thetas_f = (
            lambda mu: mu[1],
        )
        self._thetas_f = thetas_f
        operators_f = (
            ufl.inner(1, v) * ds(1),
        )
        self._operators_f = operators_f
        self._operators_f_action = tuple(
            rbnicsx.backends.linear_form_action(operator_f) for operator_f in operators_f)
        assert len(thetas_f) == len(operators_f)
        self._Q_f = len(thetas_f)
        reduced_operators_f: typing.Dict[int, rbnicsx.online.TensorsList] = dict()
        self._reduced_operators_f = reduced_operators_f
        # Define inner product using the H^1 seminorm.
        inner_product = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
        self._inner_product = inner_product
        self._inner_product_action = rbnicsx.backends.bilinear_form_action(
            inner_product, part="real")
        # Define the energy inner product
        energy_inner_product = 0.5 * (problem.bilinear_form + ufl.adjoint(problem.bilinear_form))
        self._energy_inner_product = energy_inner_product
        self._energy_inner_product_action = rbnicsx.backends.bilinear_form_action(
            energy_inner_product, part="real")

    @property
    def basis_functions(self) -> rbnicsx.backends.FunctionsList:
        """Return the basis functions of the reduced problem."""
        return self._basis_functions

    @property
    def inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product

    @property
    def inner_product_action(self) -> typing.Callable:
        """
        Return the action of the bilinear form that defines the inner product associated to this reduced problem.

        This inner product is used to define the notion of orthogonality employed during the offline stage.
        """
        return self._inner_product_action

    @property
    def energy_inner_product_form(self) -> ufl.Form:
        """
        Return the bilinear form that defines the energy inner product associated to this reduced problem.

        For elliptic coercive problems this inner product is used to evaluate the norm of the error.
        """
        return self._energy_inner_product

    def preassemble_reduced_operators(self, N: int) -> None:
        """Preassemble reduced operators in the separable expansion."""
        # Preassemble left-hand side reduced operators
        reduced_operators_a_N = rbnicsx.online.TensorsList((N, N))
        reduced_operators_a_N.extend([
            rbnicsx.backends.project_matrix(operator_a_action, self._basis_functions[:N])
            for operator_a_action in self._operators_a_action])
        assert N not in self._reduced_operators_a
        self._reduced_operators_a[N] = reduced_operators_a_N
        # Preassemble right-hand side reduced operators
        reduced_operators_f_N = rbnicsx.online.TensorsList(N)
        reduced_operators_f_N.extend([
            rbnicsx.backends.project_vector(operator_f_action, self._basis_functions[:N])
            for operator_f_action in self._operators_f_action])
        assert N not in self._reduced_operators_f
        self._reduced_operators_f[N] = reduced_operators_f_N

    def _assemble_matrix(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side online matrix of dimension N."""
        thetas_a_mu = rbnicsx.online.create_vector(self._Q_a)
        thetas_a_mu[:] = [theta_a(mu) for theta_a in self._thetas_a]
        return self._reduced_operators_a[N] * thetas_a_mu

    def _assemble_vector(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side online vector of dimension N."""
        thetas_f_mu = rbnicsx.online.create_vector(self._Q_f)
        thetas_f_mu[:] = [theta_f(mu) for theta_f in self._thetas_f]
        return self._reduced_operators_f[N] * thetas_f_mu

    def solve(self, mu: np.typing.NDArray[np.float64], N: int) -> petsc4py.PETSc.Vec:
        """Solve the online problem."""
        reduced_solution = rbnicsx.online.create_vector(N)
        if N > 0:
            A = self._assemble_matrix(mu, N)
            F = self._assemble_vector(mu, N)
            ksp = petsc4py.PETSc.KSP()
            ksp.create(mesh.comm)
            ksp.setOperators(A)
            ksp.setType("preonly")
            ksp.getPC().setType("lu")
            ksp.setFromOptions()
            ksp.solve(F, reduced_solution)
        return reduced_solution

    def reconstruct_solution(self, reduced_solution: petsc4py.PETSc.Vec) -> dolfinx.fem.Function:
        """Reconstructed reduced solution on the high fidelity space."""
        return self.basis_functions[:reduced_solution.size] * reduced_solution

    def compute_pointwise_error(
        self, solution: dolfinx.fem.Function, reduced_solution: petsc4py.PETSc.Vec
    ) -> dolfinx.fem.Function:
        """Compute the pointwise error and its norm."""
        reconstructed_reduced_solution = self.reconstruct_solution(reduced_solution)
        assert solution.function_space == reconstructed_reduced_solution.function_space
        error = dolfinx.fem.Function(solution.function_space)
        with error.vector.localForm() as error_local, solution.vector.localForm() as solution_local, \
                reconstructed_reduced_solution.vector.localForm() as reduced_solution_local:
            error_local[:] = solution_local.array - reduced_solution_local.array
        return error

    def compute_error_norm(self, error: dolfinx.fem.Function) -> petsc4py.PETSc.RealType:
        """Compute the norm of the error using the natural inner product."""
        return np.sqrt(self._inner_product_action(error)(error))

    def compute_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], error: dolfinx.fem.Function
    ) -> petsc4py.PETSc.RealType:
        """Compute the norm of the error using the energy inner product."""
        self._mu_symb.value[:] = mu
        return np.sqrt(self._energy_inner_product_action(error)(error))

In [ ]:
%%run_if pod_galerkin_efficient
class PODGReducedProblem(ProjectionBasedReducedProblem):
    """Specialize the projection-based problem to basis generation with RB."""

    pass  # No further specialization needed

In [ ]:
%%run_if reduced_basis_efficient
class RBReducedProblem(ProjectionBasedReducedProblem):
    """Specialize the projection-based problem to basis generation with POD."""

    def __init__(self, problem: Problem, stability_factor_lower_bound: object) -> None:
        super().__init__(problem)
        # Store object that evaluates the stability factor lower bound
        self._stability_factor_lower_bound = stability_factor_lower_bound
        # Prepare storage for Riesz representation
        self._riesz_representation_problem = RieszRepresentationProblem(problem, self)
        self._riesz_a = [rbnicsx.backends.FunctionsList(problem.function_space) for q_a in range(self._Q_a)]
        self._riesz_f = [rbnicsx.backends.FunctionsList(problem.function_space) for q_f in range(self._Q_f)]
        # Prepare compiled forms for Riesz representation
        u, _ = problem.trial_and_test
        riesz_a_rhs_placeholder = dolfinx.fem.Function(problem.function_space)
        self._riesz_a_rhs_cpp = [
            dolfinx.fem.form(ufl.replace(- operator_a, {u: riesz_a_rhs_placeholder}))
            for operator_a in self._operators_a]
        self._riesz_a_rhs_placeholder = riesz_a_rhs_placeholder
        self._riesz_f_rhs_cpp = [dolfinx.fem.form(operator_f) for operator_f in self._operators_f]
        # Prepare storage for error estimation operators
        self._error_estimation_operator_ff: rbnicsx.online.TensorsArray = None
        self._error_estimation_operator_af: typing.Dict[int, rbnicsx.online.TensorsArray] = dict()
        self._error_estimation_operator_aa: typing.Dict[int, rbnicsx.online.TensorsArray] = dict()

    def compute_riesz_representation(self, N: int) -> None:
        """Compute Riesz representation of operators in the separable expansion."""
        # Compute Riesz representation of the right-hand side operators.
        if N == 0:
            # The representation is independent on N, and thus needs to be computed only
            # the first time that this method is called.
            for (riesz_f_rhs_cpp, riesz_f) in zip(self._riesz_f_rhs_cpp, self._riesz_f):
                riesz_f.append(self._riesz_representation_problem.represent(riesz_f_rhs_cpp))
        else:
            for riesz_f in self._riesz_f:
                assert len(riesz_f) == 1
        # Compute Riesz representation of the left-hand side operators
        if N > 0:
            for (riesz_a_rhs_cpp, riesz_a) in zip(self._riesz_a_rhs_cpp, self._riesz_a):
                assert len(riesz_a) == N - 1
                assert len(self._basis_functions) == N
                with self._basis_functions[N - 1].vector.localForm() as basis_local, \
                        self._riesz_a_rhs_placeholder.vector.localForm() as riesz_a_rhs_placeholder_local:
                    basis_local.copy(riesz_a_rhs_placeholder_local)
                riesz_a.append(self._riesz_representation_problem.represent(riesz_a_rhs_cpp))

    def preassemble_error_estimation_operators(self, N: int) -> None:
        """Preassemble error estimation operators associated to the separable expansion."""
        # Preassemble error estimation operators associated to the (right-hand side, right-hand side) products.
        if N == 0:
            # The error estimation is independent on N, and thus needs to be computed only
            # the first time that this method is called.
            error_estimation_operator_ff = rbnicsx.online.TensorsArray((1, 1), (self._Q_f, self._Q_f))
            for q_f in range(self._Q_f):
                for qq_f in range(self._Q_f):
                    error_estimation_operator_ff[q_f, qq_f] = rbnicsx.backends.project_matrix(
                        self._inner_product_action, (self._riesz_f[q_f], self._riesz_f[qq_f]))
            self._error_estimation_operator_ff = error_estimation_operator_ff
        # Preassemble error estimation operators associated to the (left-hand side, right-hand side) products.
        if N > 0:
            error_estimation_operator_af_N = rbnicsx.online.TensorsArray((N, 1), (self._Q_a, self._Q_f))
            for q_a in range(self._Q_a):
                for q_f in range(self._Q_f):
                    error_estimation_operator_af_N[q_a, q_f] = rbnicsx.backends.project_matrix(
                        self._inner_product_action, (self._riesz_a[q_a], self._riesz_f[q_f]))
            self._error_estimation_operator_af[N] = error_estimation_operator_af_N
        # Preassemble error estimation operators associated to the (left-hand side, left-hand side) products.
        if N > 0:
            error_estimation_operator_aa_N = rbnicsx.online.TensorsArray((N, N), (self._Q_a, self._Q_a))
            for q_a in range(self._Q_a):
                for qq_a in range(self._Q_a):
                    error_estimation_operator_aa_N[q_a, qq_a] = rbnicsx.backends.project_matrix(
                        self._inner_product_action, (self._riesz_a[q_a], self._riesz_a[qq_a]))
            self._error_estimation_operator_aa[N] = error_estimation_operator_aa_N

    def get_residual_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Compute the squared natural norm of the residual."""
        thetas_a_mu = rbnicsx.online.create_vector(self._Q_a)
        thetas_a_mu[:] = [theta_a(mu) for theta_a in self._thetas_a]
        thetas_f_mu = rbnicsx.online.create_vector(self._Q_f)
        thetas_f_mu[:] = [theta_f(mu) for theta_f in self._thetas_f]
        N = reduced_solution.size
        residual_norm_squared = (
            self._error_estimation_operator_ff.contraction(thetas_f_mu, thetas_f_mu))
        if N > 0:
            residual_norm_squared += (
                2 * self._error_estimation_operator_af[N].contraction(thetas_a_mu, thetas_f_mu, reduced_solution)
                + self._error_estimation_operator_aa[N].contraction(
                    thetas_a_mu, thetas_a_mu, reduced_solution, reduced_solution))
        if np.issubdtype(petsc4py.PETSc.ScalarType, np.complexfloating):
            assert np.isclose(residual_norm_squared.imag, 0)
            residual_norm_squared = residual_norm_squared.real
        if residual_norm_squared < 0:
            assert np.isclose(residual_norm_squared, 0)
            residual_norm_squared = abs(residual_norm_squared)
        return np.sqrt(residual_norm_squared)

    def estimate_error_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate the norm of the error using the natural inner product."""
        return self.get_residual_norm(mu, reduced_solution) / (
            self._stability_factor_lower_bound.evaluate(mu))

    def estimate_error_energy_norm(
        self, mu: np.typing.NDArray[np.float64], reduced_solution: petsc4py.PETSc.Vec
    ) -> petsc4py.PETSc.RealType:
        """Estimate the norm of the error using the energy inner product."""
        return self.get_residual_norm(mu, reduced_solution) / np.sqrt(
            self._stability_factor_lower_bound.evaluate(mu))

## 4. Training

In [ ]:
def generate_training_set() -> np.typing.NDArray[np.float64]:
    """Generate an equispaced training set using numpy."""
    training_set_0 = np.linspace(0.1, 10.0, 10)
    training_set_1 = np.linspace(0.0, 1.0, 10)
    training_set = np.dstack(np.meshgrid(training_set_0, training_set_1)).reshape(-1, 2)
    return training_set


training_set = rbnicsx.io.on_rank_zero(mesh.comm, generate_training_set)
assert training_set.shape == (100, 2)

In [ ]:
Nmax = 4

In [ ]:
%%run_if pod_galerkin_inefficient
print(rbnicsx.io.TextBox("POD-Galerkin offline phase begins", fill="="))
print("")

print("set up snapshots matrix")
snapshots_matrix = rbnicsx.backends.FunctionsList(problem.function_space)

print("set up reduced problem")
reduced_problem = PODGInefficientReducedProblem(problem)

print("")

for (mu_index, mu) in enumerate(training_set):
    print(rbnicsx.io.TextLine(str(mu_index), fill="#"))

    print("high fidelity solve for mu =", mu)
    snapshot = problem.solve(mu)

    print("update snapshots matrix")
    snapshots_matrix.append(snapshot)

    print("")

print(rbnicsx.io.TextLine("perform POD", fill="#"))
eigenvalues, modes, _ = rbnicsx.backends.proper_orthogonal_decomposition(
    snapshots_matrix, reduced_problem.inner_product_action, N=Nmax, tol=0.0)
reduced_problem.basis_functions.extend(modes)
print("")

print(rbnicsx.io.TextBox("POD-Galerkin offline phase ends", fill="="))

In [ ]:
%%run_if pod_galerkin_efficient
print(rbnicsx.io.TextBox("POD-Galerkin offline phase begins", fill="="))
print("")

print("set up snapshots matrix")
snapshots_matrix = rbnicsx.backends.FunctionsList(problem.function_space)

print("set up reduced problem")
reduced_problem = PODGReducedProblem(problem)

print("")

for (mu_index, mu) in enumerate(training_set):
    print(rbnicsx.io.TextLine(str(mu_index), fill="#"))

    print("high fidelity solve for mu =", mu)
    snapshot = problem.solve(mu)

    print("update snapshots matrix")
    snapshots_matrix.append(snapshot)

    print("")

print(rbnicsx.io.TextLine("perform POD", fill="#"))
eigenvalues, modes, _ = rbnicsx.backends.proper_orthogonal_decomposition(
    snapshots_matrix, reduced_problem.inner_product_action, N=Nmax, tol=0.0)
reduced_problem.basis_functions.extend(modes)
print("")

print("preassemble reduced operators")
for N in range(1, Nmax + 1):
    reduced_problem.preassemble_reduced_operators(N)
print("")

print(rbnicsx.io.TextBox("POD-Galerkin offline phase ends", fill="="))

In [ ]:
%%run_if pod_galerkin_efficient, pod_galerkin_inefficient
positive_eigenvalues = np.where(eigenvalues > 0., eigenvalues, np.nan)
singular_values = np.sqrt(positive_eigenvalues)

In [ ]:
%%run_if pod_galerkin_efficient, pod_galerkin_inefficient
fig = go.Figure()
fig.add_scatter(
    x=np.arange(singular_values.shape[0]), y=singular_values / singular_values[0], mode="markers+lines")
fig.update_layout(title="Normalized singular values")
fig.update_xaxes(title_text="N"),
fig.update_yaxes(title_text=r"$\frac{\sigma_N}{\sigma_0}$", type="log", exponentformat="power")
fig.show()

In [ ]:
%%run_if pod_galerkin_efficient, pod_galerkin_inefficient
assert np.all(rbnicsx.test.order_of_magnitude(eigenvalues[:Nmax]) == [2, 1, -4, -8])

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
def greedy(reduced_problem: object, N: int) -> typing.Tuple[int, petsc4py.PETSc.RealType]:
    """Carry out a greedy search of the training set."""
    error_estimators = np.full(len(training_set), fill_value=np.nan)
    for (mu_index, mu) in enumerate(training_set):
        reduced_solution = reduced_problem.solve(mu, N)
        error_estimators[mu_index] = reduced_problem.estimate_error_energy_norm(mu, reduced_solution)
    greedy_index = np.argmax(error_estimators)
    return greedy_index, error_estimators[greedy_index]

In [ ]:
%%run_if reduced_basis_inefficient
print(rbnicsx.io.TextBox("RB offline phase begins", fill="="))
print("")

print("set up reduced problem")
reduced_problem = RBInefficientReducedProblem(problem, stability_factor_lower_bound)

print("determine initial parameter")
initial_greedy_index, initial_absolute_error_estimator = greedy(reduced_problem, 0)
greedy_indices = [initial_greedy_index]
absolute_error_estimators = [initial_absolute_error_estimator]
print("\tinitial maximum absolute error estimator over training set =", initial_absolute_error_estimator)
mu = training_set[initial_greedy_index]
print("\tinitial parameter = ", mu)
print("\tinitial parameter index = ", initial_greedy_index)

print("")

N = 0
while N < Nmax:
    print(rbnicsx.io.TextLine("N = " + str(N), fill="#"))

    print("high fidelity solve for mu =", mu)
    snapshot = problem.solve(mu)

    print("update basis functions")
    rbnicsx.backends.gram_schmidt(
        reduced_problem.basis_functions, snapshot, reduced_problem.inner_product_action)
    N += 1

    print("reduced order solve for consistency check")
    reduced_snapshot = reduced_problem.solve(mu, N)
    error_snapshot = reduced_problem.compute_pointwise_error(snapshot, reduced_snapshot)
    print(
        "\tabsolute error for current parameter =",
        reduced_problem.compute_error_energy_norm(mu, error_snapshot))
    print(
        "\tabsolute error estimator for current parameter =",
        reduced_problem.estimate_error_energy_norm(mu, reduced_snapshot))

    print("determine next parameter")
    greedy_index, absolute_error_estimator = greedy(reduced_problem, N)
    greedy_indices.append(greedy_index)
    absolute_error_estimators.append(absolute_error_estimator)
    print("\tmaximum absolute error estimator over training set =", absolute_error_estimator)
    print(
        "\tmaximum relative error estimator over training set =",
        absolute_error_estimator / initial_absolute_error_estimator)
    mu = training_set[greedy_index]
    print("\tnext parameter = ", mu)
    print("\tnext parameter index = ", greedy_index)

    print("")

print(rbnicsx.io.TextBox("RB offline phase ends", fill="="))

In [ ]:
%%run_if reduced_basis_efficient
print(rbnicsx.io.TextBox("RB offline phase begins", fill="="))
print("")

print("set up reduced problem")
reduced_problem = RBReducedProblem(problem, stability_factor_lower_bound)

print("preassemble error estimation operators")
reduced_problem.compute_riesz_representation(0)
reduced_problem.preassemble_error_estimation_operators(0)

print("determine initial parameter")
initial_greedy_index, initial_absolute_error_estimator = greedy(reduced_problem, 0)
greedy_indices = [initial_greedy_index]
absolute_error_estimators = [initial_absolute_error_estimator]
print("\tinitial maximum absolute error estimator over training set =", initial_absolute_error_estimator)
mu = training_set[initial_greedy_index]
print("\tinitial parameter = ", mu)
print("\tinitial parameter index = ", initial_greedy_index)

print("")

N = 0
while N < Nmax:
    print(rbnicsx.io.TextLine("N = " + str(N), fill="#"))

    print("high fidelity solve for mu =", mu)
    snapshot = problem.solve(mu)

    print("update basis functions")
    rbnicsx.backends.gram_schmidt(
        reduced_problem.basis_functions, snapshot, reduced_problem.inner_product_action)
    N += 1

    print("preassemble reduced operators")
    reduced_problem.preassemble_reduced_operators(N)

    print("preassemble error estimation operators")
    reduced_problem.compute_riesz_representation(N)
    reduced_problem.preassemble_error_estimation_operators(N)

    print("reduced order solve for consistency check")
    reduced_snapshot = reduced_problem.solve(mu, N)
    error_snapshot = reduced_problem.compute_pointwise_error(snapshot, reduced_snapshot)
    print(
        "\tabsolute error for current parameter =",
        reduced_problem.compute_error_energy_norm(mu, error_snapshot))
    print(
        "\tabsolute error estimator for current parameter =",
        reduced_problem.estimate_error_energy_norm(mu, reduced_snapshot))

    print("determine next parameter")
    greedy_index, absolute_error_estimator = greedy(reduced_problem, N)
    greedy_indices.append(greedy_index)
    absolute_error_estimators.append(absolute_error_estimator)
    print("\tmaximum absolute error estimator over training set =", absolute_error_estimator)
    print(
        "\tmaximum relative error estimator over training set =",
        absolute_error_estimator / initial_absolute_error_estimator)
    mu = training_set[greedy_index]
    print("\tnext parameter = ", mu)
    print("\tnext parameter index = ", greedy_index)

    print("")

print(rbnicsx.io.TextBox("RB offline phase ends", fill="="))

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
fig = go.Figure()
fig.add_scatter(
    x=np.arange(N + 1),
    y=[e / absolute_error_estimators[0] for e in absolute_error_estimators],
    mode="markers+lines")
fig.update_layout(title="Greedy maximum error estimator")
fig.update_xaxes(title_text="N"),
fig.update_yaxes(title_text=r"$\Delta_N$", type="log", exponentformat="power")
fig.show()

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
assert len(absolute_error_estimators) == Nmax + 1
assert np.all(rbnicsx.test.order_of_magnitude(absolute_error_estimators) == [0, 0, -2, -5, -6])

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
not_greedy_indices = np.setdiff1d(np.arange(len(training_set)), greedy_indices)
fig = go.Figure()
fig.add_scatter(
    x=training_set[not_greedy_indices, 0], y=training_set[not_greedy_indices, 1],
    mode="markers", marker_color="blue", name="not selected")
fig.add_scatter(
    x=training_set[greedy_indices, 0], y=training_set[greedy_indices, 1],
    mode="markers", marker_color="red", name="selected")
fig.update_layout(title="Greedy selected parameters")
fig.update_xaxes(title_text="mu[0]"),
fig.update_yaxes(title_text="mu[1]")
fig.show()

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
assert len(greedy_indices) == Nmax + 1
assert np.all(greedy_indices == [90, 99, 92, 91, 95])

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
multiphenicsx.io.plot_scalar_field(reduced_problem.basis_functions[0], "first basis solution")

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
multiphenicsx.io.plot_scalar_field(reduced_problem.basis_functions[1], "second basis solution")

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
multiphenicsx.io.plot_scalar_field(reduced_problem.basis_functions[2], "third basis solution")

## 5. Testing

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
reduced_solution = reduced_problem.solve(mu_solve, Nmax)

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
reconstructed_reduced_solution = reduced_problem.reconstruct_solution(reduced_solution)
multiphenicsx.io.plot_scalar_field(reconstructed_reduced_solution, "reconstructed reduced solution")

In [ ]:
%%run_if reduced_basis_inefficient, reduced_basis_efficient, pod_galerkin_inefficient, pod_galerkin_efficient
error = reduced_problem.compute_pointwise_error(solution, reduced_solution)
multiphenicsx.io.plot_scalar_field(error, "pointwise error")

In [ ]:
%%run_if pod_galerkin_efficient, pod_galerkin_inefficient
error_norm = reduced_problem.compute_error_norm(error)
error_energy_norm = reduced_problem.compute_error_energy_norm(mu_solve, error)
print("Natural norm of the error", error_norm)
print("Energy norm of the error", error_energy_norm)

In [ ]:
%%run_if pod_galerkin_efficient, pod_galerkin_inefficient
assert rbnicsx.test.order_of_magnitude(error_norm) == -9
assert rbnicsx.test.order_of_magnitude(error_energy_norm) == -8

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
error_norm = reduced_problem.compute_error_norm(error)
error_norm_estimator = reduced_problem.estimate_error_norm(mu_solve, reduced_solution)
error_energy_norm = reduced_problem.compute_error_energy_norm(mu_solve, error)
error_energy_norm_estimator = reduced_problem.estimate_error_energy_norm(mu_solve, reduced_solution)
print("Natural norm of the error", error_norm, "estimated as", error_norm_estimator)
print("Energy norm of the error", error_energy_norm, "estimated as", error_energy_norm_estimator)

In [ ]:
%%run_if reduced_basis_efficient, reduced_basis_inefficient
assert rbnicsx.test.order_of_magnitude(error_norm) == -7
assert rbnicsx.test.order_of_magnitude(error_norm_estimator) in (-7, -6)
assert rbnicsx.test.order_of_magnitude(error_energy_norm) == -7
assert rbnicsx.test.order_of_magnitude(error_energy_norm_estimator) in (-7, -6)